In [8]:
from db import get_engine, df_from_sql
engine = get_engine()

sql_count = """
SELECT COUNT(*) AS n
FROM public.profiles
WHERE linkedin_url ~* '^(https?://)www\\.public\\.com/';
"""
sql_sample = """
SELECT profile_id, linkedin_url
FROM public.profiles
WHERE linkedin_url ~* '^(https?://)www\\.public\\.com/'
ORDER BY profile_id
LIMIT 20;
"""

print("🔎 Conteo de filas con dominio incorrecto:")
print(df_from_sql(sql_count, engine))

print("\n🧪 Muestra (20 filas):")
df_from_sql(sql_sample, engine)


🔎 Conteo de filas con dominio incorrecto:
     n
0  445

🧪 Muestra (20 filas):


,profile_id,linkedin_url
0,1,https://www.public.com/in/iara-bessone-1b9047186
1,2,https://www.public.com/in/geraldine-beskow-b96...
2,3,https://www.public.com/in/agustinbesso
3,4,https://www.public.com/in/agustinabetz
4,5,https://www.public.com/in/santiagobianciotti
5,6,https://www.public.com/in/gonzaloblaconasainz
6,7,https://www.public.com/in/pbiondi1989
7,8,https://www.public.com/in/fdbianco
8,9,https://www.public.com/in/meli-bircher
9,10,https://www.public.com/in/blancoezequiel


In [9]:
import pandas as pd
import os, datetime as dt

backup_dir = "../backups"
os.makedirs(backup_dir, exist_ok=True)
ts = dt.datetime.now().strftime("%Y%m%d_%H%M%S")

sql_backup = """
SELECT *
FROM public.profiles
WHERE linkedin_url ~* '^(https?://)www\\.public\\.com/';
"""
df_backup = df_from_sql(sql_backup, engine)
backup_path = os.path.join(backup_dir, f"profiles_public_domain_backup_{ts}.csv")
df_backup.to_csv(backup_path, index=False, encoding="utf-8")
backup_path


'../backups\\profiles_public_domain_backup_20251030_011307.csv'

In [11]:
from sqlalchemy import text

update_sql = text("""
UPDATE public.profiles
SET linkedin_url = regexp_replace(
    linkedin_url,
    '^(https?://)www\\.public\\.com/',
    '\\1www.linkedin.com/'
)
WHERE linkedin_url ~* '^(https?://)www\\.public\\.com/'
RETURNING profile_id, linkedin_url;
""")

updated = []
with engine.begin() as conn:  # transacción
    res = conn.execute(update_sql)
    updated = res.mappings().all()

print(f"✅ Filas actualizadas: {len(updated)}")
# Mostrar algunas filas resultantes
import pandas as pd
pd.DataFrame(updated).head(20)


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "profiles_linkedin_url_key"
DETAIL:  Key (linkedin_url)=(https://www.linkedin.com/in/iara-bessone-1b9047186) already exists.

[SQL: 
UPDATE public.profiles
SET linkedin_url = regexp_replace(
    linkedin_url,
    '^(https?://)www\.public\.com/',
    '\1www.linkedin.com/'
)
WHERE linkedin_url ~* '^(https?://)www\.public\.com/'
RETURNING profile_id, linkedin_url;
]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [12]:
preview_pairs_sql = """
WITH bad AS (
  SELECT
    rtrim(lower(linkedin_url), '/')            AS bad_url_norm,
    rtrim(lower(
      regexp_replace(linkedin_url,
        '^(https?://)www\\.public\\.com/',
        '\\1www.linkedin.com/'
      )
    ), '/')                                   AS target_url_norm
  FROM public.profiles
  WHERE linkedin_url ~* '^(https?://)www\\.public\\.com/'
)
SELECT
  p_bad.linkedin_url         AS will_update_from,
  p_dup.linkedin_url         AS existing_target_conflict
FROM bad b
JOIN public.profiles p_bad
  ON rtrim(lower(p_bad.linkedin_url), '/') = b.bad_url_norm
JOIN public.profiles p_dup
  ON rtrim(lower(p_dup.linkedin_url), '/') = b.target_url_norm
ORDER BY p_bad.linkedin_url
LIMIT 50;
"""
df_from_sql(preview_pairs_sql, engine)


,will_update_from,existing_target_conflict
0,https://www.public.com/in/1500336,https://www.linkedin.com/in/1500336
1,https://www.public.com/in/adib-abuslaiman-7901...,https://www.linkedin.com/in/adib-abuslaiman-79...
2,https://www.public.com/in/adrian-benavides,https://www.linkedin.com/in/adrian-benavides
3,https://www.public.com/in/agjosefina,https://www.linkedin.com/in/agjosefina
4,https://www.public.com/in/agostina-buffagni,https://www.linkedin.com/in/agostina-buffagni
5,https://www.public.com/in/agustina-beltramino-...,https://www.linkedin.com/in/agustina-beltramin...
6,https://www.public.com/in/agustinabetz,https://www.linkedin.com/in/agustinabetz
7,https://www.public.com/in/agustin-alonso-canto...,https://www.linkedin.com/in/agustin-alonso-can...
8,https://www.public.com/in/agustin-avalos-tabasso,https://www.linkedin.com/in/agustin-avalos-tab...
9,https://www.public.com/in/agustinbesso,https://www.linkedin.com/in/agustinbesso


In [15]:
from db import get_engine, df_from_sql
engine = get_engine()

preview_pairs = """
WITH pairs AS (
  SELECT
    keep.profile_id  AS keep_id,   -- conservamos esta (tiene public.com)
    drop_.profile_id AS drop_id,   -- borraremos esta (ya tiene linkedin.com)
    keep.linkedin_url AS keep_url_before,
    drop_.linkedin_url AS drop_url_existing
  FROM public.profiles keep
  JOIN public.profiles drop_
    ON rtrim(lower(
         regexp_replace(keep.linkedin_url,
           '^(https?://)www\\.public\\.com/', '\\1www.linkedin.com/'
         )
       ), '/') = rtrim(lower(drop_.linkedin_url), '/')
  WHERE keep.linkedin_url ~* '^(https?://)www\\.public\\.com/'
)
SELECT * FROM pairs
ORDER BY keep_id
LIMIT 50;
"""
df_from_sql(preview_pairs, engine)


,keep_id,drop_id,keep_url_before,drop_url_existing
0,1,3337,https://www.public.com/in/iara-bessone-1b9047186,https://www.linkedin.com/in/iara-bessone-1b904...
1,2,3335,https://www.public.com/in/geraldine-beskow-b96...,https://www.linkedin.com/in/geraldine-beskow-b...
2,3,3336,https://www.public.com/in/agustinbesso,https://www.linkedin.com/in/agustinbesso
3,4,3338,https://www.public.com/in/agustinabetz,https://www.linkedin.com/in/agustinabetz
4,5,3340,https://www.public.com/in/santiagobianciotti,https://www.linkedin.com/in/santiagobianciotti
5,6,3348,https://www.public.com/in/gonzaloblaconasainz,https://www.linkedin.com/in/gonzaloblaconasainz
6,7,3344,https://www.public.com/in/pbiondi1989,https://www.linkedin.com/in/pbiondi1989
7,8,3341,https://www.public.com/in/fdbianco,https://www.linkedin.com/in/fdbianco
8,9,3345,https://www.public.com/in/meli-bircher,https://www.linkedin.com/in/meli-bircher
9,10,3349,https://www.public.com/in/blancoezequiel,https://www.linkedin.com/in/blancoezequiel


In [16]:
from sqlalchemy import text
from db import get_engine, df_from_sql

engine = get_engine()

with engine.begin() as conn:
    # 1) Pares keep/drop (keep = public.com, drop = linkedin.com)
    conn.execute(text("CREATE TEMP TABLE pairs(keep_id bigint, drop_id bigint) ON COMMIT DROP;"))

    conn.execute(text("""
        INSERT INTO pairs(keep_id, drop_id)
        SELECT
          keep.profile_id  AS keep_id,
          drop_.profile_id AS drop_id
        FROM public.profiles keep
        JOIN public.profiles drop_
          ON rtrim(lower(
               regexp_replace(keep.linkedin_url,
                 '^(https?://)www\\.public\\.com/', '\\1www.linkedin.com/'
               )
             ), '/') = rtrim(lower(drop_.linkedin_url), '/')
        WHERE keep.linkedin_url ~* '^(https?://)www\\.public\\.com/';
    """))

    # 2) Reasignar TODAS las FKs que referencian a public.profiles(profile_id)
    conn.execute(text("""
    DO $$
    DECLARE
      r RECORD;
    BEGIN
      FOR r IN
        SELECT
          n.nspname         AS fk_schema,
          c.relname         AS fk_table,
          a.attname         AS fk_column
        FROM pg_constraint ct
        JOIN pg_class c           ON c.oid = ct.conrelid
        JOIN pg_namespace n       ON n.oid = c.relnamespace
        JOIN pg_class cref        ON cref.oid = ct.confrelid
        JOIN pg_namespace nref    ON nref.oid = cref.relnamespace
        JOIN unnest(ct.conkey) WITH ORDINALITY AS ck(attnum, ord)
             JOIN pg_attribute a  ON a.attrelid = ct.conrelid AND a.attnum = ck.attnum
        WHERE ct.contype = 'f'
          AND nref.nspname = 'public'
          AND cref.relname = 'profiles'
      LOOP
        EXECUTE format($F$
          UPDATE %I.%I t
          SET %I = p.keep_id
          FROM pairs p
          WHERE t.%I = p.drop_id
        $F$, r.fk_schema, r.fk_table, r.fk_column, r.fk_column);
      END LOOP;
    END$$;
    """))

    # 3) Borrar duplicadas (las que ya tenían linkedin.com)
    del_res = conn.execute(text("""
        DELETE FROM public.profiles pr
        USING pairs p
        WHERE pr.profile_id = p.drop_id;
    """))

    # 4) Corregir dominio en las filas conservadas
    upd_res = conn.execute(text("""
        UPDATE public.profiles pr
        SET linkedin_url = regexp_replace(pr.linkedin_url,
                           '^(https?://)www\\.public\\.com/', '\\1www.linkedin.com/')
        FROM pairs p
        WHERE pr.profile_id = p.keep_id;
    """))

print("✅ Fusión completada.")


ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "WHERE"
LINE 18:         WHERE ct.contype = 'f'
                 ^

[SQL: 
    DO $$
    DECLARE
      r RECORD;
    BEGIN
      FOR r IN
        SELECT
          n.nspname         AS fk_schema,
          c.relname         AS fk_table,
          a.attname         AS fk_column
        FROM pg_constraint ct
        JOIN pg_class c           ON c.oid = ct.conrelid
        JOIN pg_namespace n       ON n.oid = c.relnamespace
        JOIN pg_class cref        ON cref.oid = ct.confrelid
        JOIN pg_namespace nref    ON nref.oid = cref.relnamespace
        JOIN unnest(ct.conkey) WITH ORDINALITY AS ck(attnum, ord)
             JOIN pg_attribute a  ON a.attrelid = ct.conrelid AND a.attnum = ck.attnum
        WHERE ct.contype = 'f'
          AND nref.nspname = 'public'
          AND cref.relname = 'profiles'
      LOOP
        EXECUTE format($F$
          UPDATE %%I.%%I t
          SET %%I = p.keep_id
          FROM pairs

In [17]:
from sqlalchemy import text
from db import get_engine, df_from_sql
import pandas as pd

engine = get_engine()

with engine.begin() as conn:
    # Pares (keep = public.com, drop = linkedin.com)
    conn.execute(text("CREATE TEMP TABLE pairs(keep_id bigint, drop_id bigint) ON COMMIT DROP;"))
    conn.execute(text("""
        INSERT INTO pairs(keep_id, drop_id)
        SELECT
          keep.profile_id  AS keep_id,
          drop_.profile_id AS drop_id
        FROM public.profiles keep
        JOIN public.profiles drop_
          ON rtrim(lower(
               regexp_replace(keep.linkedin_url,
                 '^(https?://)www\\.public\\.com/', '\\1www.linkedin.com/'
               )
             ), '/') = rtrim(lower(drop_.linkedin_url), '/')
        WHERE keep.linkedin_url ~* '^(https?://)www\\.public\\.com/';
    """))

# Listar tablas/columnas con FK → profiles(profile_id)
fk_list_sql = """
SELECT
  n.nspname  AS fk_schema,
  c.relname  AS fk_table,
  a.attname  AS fk_column
FROM pg_constraint con
JOIN pg_class      c  ON c.oid = con.conrelid
JOIN pg_namespace  n  ON n.oid = c.relnamespace
JOIN pg_class      cr ON cr.oid = con.confrelid
JOIN pg_namespace  nr ON nr.oid = cr.relnamespace
JOIN pg_attribute  a  ON a.attrelid = con.conrelid AND a.attnum = ANY (con.conkey)
WHERE con.contype   = 'f'
  AND nr.nspname    = 'public'
  AND cr.relname    = 'profiles';
"""

df_fks = df_from_sql(fk_list_sql, engine)
df_fks


,fk_schema,fk_table,fk_column
0,public,educations,profile_id
1,public,experiences,profile_id
2,public,profile_languages,profile_id
3,public,profile_skills,profile_id


In [18]:
from sqlalchemy import text

engine = get_engine()
updates_done = []

with engine.begin() as conn:
    for _, row in df_fks.iterrows():
        schema = row["fk_schema"]
        table  = row["fk_table"]
        col    = row["fk_column"]

        # Construimos SQL con identificadores entre comillas para seguridad
        sql = f'''
            UPDATE "{schema}"."{table}" t
            SET "{col}" = p.keep_id
            FROM pairs p
            WHERE t."{col}" = p.drop_id;
        '''
        res = conn.execute(text(sql))
        updates_done.append((f'{schema}.{table}.{col}', res.rowcount))

updates_done[:10], len(updates_done)


ProgrammingError: (psycopg2.errors.UndefinedTable) relation "pairs" does not exist
LINE 4:             FROM pairs p
                         ^

[SQL: 
            UPDATE "public"."educations" t
            SET "profile_id" = p.keep_id
            FROM pairs p
            WHERE t."profile_id" = p.drop_id;
        ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [20]:
from sqlalchemy import text
import pandas as pd
from db import get_engine

engine = get_engine()

with engine.begin() as conn:
    # 1) Pairs dentro de la MISMA conexión
    conn.execute(text("""
        CREATE TEMP TABLE pairs(keep_id bigint, drop_id bigint) ON COMMIT PRESERVE ROWS;
    """))

    conn.execute(text("""
        INSERT INTO pairs(keep_id, drop_id)
        SELECT
          keep.profile_id  AS keep_id,
          drop_.profile_id AS drop_id
        FROM public.profiles keep
        JOIN public.profiles drop_
          ON rtrim(lower(
               regexp_replace(keep.linkedin_url,
                 '^(https?://)www\\.public\\.com/', '\\1www.linkedin.com/'
               )
             ), '/') = rtrim(lower(drop_.linkedin_url), '/')
        WHERE keep.linkedin_url ~* '^(https?://)www\\.public\\.com/';
    """))

    # 2) Todas las FKs que apuntan a public.profiles(profile_id)
    fk_list_sql = """
    SELECT
      n.nspname  AS fk_schema,
      c.relname  AS fk_table,
      a.attname  AS fk_column
    FROM pg_constraint con
    JOIN pg_class      c  ON c.oid = con.conrelid
    JOIN pg_namespace  n  ON n.oid = c.relnamespace
    JOIN pg_class      cr ON cr.oid = con.confrelid
    JOIN pg_namespace  nr ON nr.oid = cr.relnamespace
    JOIN pg_attribute  a  ON a.attrelid = con.conrelid AND a.attnum = ANY (con.conkey)
    WHERE con.contype   = 'f'
      AND nr.nspname    = 'public'
      AND cr.relname    = 'profiles';
    """
    fk_rows = conn.execute(text(fk_list_sql)).mappings().all()
    df_fks = pd.DataFrame(fk_rows)

    # Helper para obtener las columnas PK de una tabla
    def get_pk_cols(schema, table):
        pk_sql = text("""
            SELECT a.attname
            FROM pg_index i
            JOIN pg_class t   ON t.oid = i.indrelid
            JOIN pg_namespace n ON n.oid = t.relnamespace
            JOIN pg_attribute a ON a.attrelid = t.oid AND a.attnum = ANY(i.indkey)
            WHERE i.indisprimary = TRUE
              AND n.nspname = :schema
              AND t.relname = :table
            ORDER BY a.attnum;
        """)
        rows = conn.execute(pk_sql, {"schema": schema, "table": table}).scalars().all()
        return rows

    # 3) Para cada tabla hija: borrar potenciales duplicados y luego actualizar
    for _, r in df_fks.iterrows():
        schema = r["fk_schema"]
        table  = r["fk_table"]
        col    = r["fk_column"]

        pk_cols = get_pk_cols(schema, table)

        # 3a) Si la PK incluye la columna FK (caso típico: profile_languages(profile_id, lang_id))
        if col in pk_cols:
            other_pk = [c for c in pk_cols if c != col]
            if other_pk:
                # Construimos condiciones de igualdad en las otras columnas PK
                eq_conds = " AND ".join([f't2."{c}" = t."{c}"' for c in other_pk])

                # Borrar las filas del drop_id que colisionarían con keep_id
                delete_dup_sql = f'''
                    DELETE FROM "{schema}"."{table}" t
                    USING pairs p
                    WHERE t."{col}" = p.drop_id
                      AND EXISTS (
                        SELECT 1
                        FROM "{schema}"."{table}" t2
                        WHERE t2."{col}" = p.keep_id
                          AND {eq_conds}
                      );
                '''
                conn.execute(text(delete_dup_sql))
        # 3b) Actualizar todas las filas restantes del drop_id -> keep_id
        update_sql = f'''
            UPDATE "{schema}"."{table}" t
            SET "{col}" = p.keep_id
            FROM pairs p
            WHERE t."{col}" = p.drop_id;
        '''
        conn.execute(text(update_sql))

    # 4) Borrar duplicadas en profiles (las "drop") y corregir dominio en las "keep"
    conn.execute(text("""
        DELETE FROM public.profiles pr
        USING pairs p
        WHERE pr.profile_id = p.drop_id;
    """))

    conn.execute(text("""
        UPDATE public.profiles pr
        SET linkedin_url = regexp_replace(pr.linkedin_url,
                           '^(https?://)www\\.public\\.com/', '\\1www.linkedin.com/')
        FROM pairs p
        WHERE pr.profile_id = p.keep_id;
    """))

    # 5) Normalizar barra final
    conn.execute(text("""
        UPDATE public.profiles
        SET linkedin_url = rtrim(linkedin_url, '/')
        WHERE linkedin_url LIKE '%/';
    """))

    # Limpieza
    conn.execute(text("DROP TABLE IF EXISTS pairs;"))

print("✅ Merge OK: FKs reasignados con deduplicación previa, duplicados borrados y URLs arregladas.")


✅ Merge OK: FKs reasignados con deduplicación previa, duplicados borrados y URLs arregladas.


In [21]:
from db import df_from_sql

# 0 con public.com
df_from_sql("""
SELECT COUNT(*) AS restantes_public
FROM public.profiles
WHERE linkedin_url ~* '^(https?://)www\\.public\\.com/';
""", engine)


,restantes_public
0,5
